# Web Scraping Data - Final

Using the steps used in the draft file, I will write a cleaner version of the code that will scrape the stats for all the teams; current and previous years. The data is saved in a csv file.

In [ ]:
#importing necessary libraries
import requests
from bs4 import BeautifulSoup

import pandas as pd
import time

In [ ]:
#Initializing our variables
years = list(range(2023, 2018, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
#Scarpping the given url for EPL data from 2023 to 2019 season
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)

<ipython-input-3-581298adf1e9>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Season"] = year
<ipython-input-3-581298adf1e9>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Team"] = team_name


In [ ]:
#Concatinating all the matches for all seasons into a single dataframe
df = pd.concat(all_matches)

In [ ]:
df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,2023,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,2023,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,2023,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,2023,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,29.0,13.0,16.4,1.0,0,0,2023,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2020-07-07,18:00,Premier League,Matchweek 34,Tue,Away,L,1,2,Watford,...,Match Report,NaN,12.0,3.0,16.9,1.0,0,0,2019,Norwich City
39,2020-07-11,12:30,Premier League,Matchweek 35,Sat,Home,L,0,4,West Ham,...,Match Report,NaN,11.0,2.0,19.8,1.0,0,0,2019,Norwich City
40,2020-07-14,20:15,Premier League,Matchweek 36,Tue,Away,L,0,1,Chelsea,...,Match Report,NaN,2.0,0.0,20.6,0.0,0,0,2019,Norwich City
41,2020-07-18,17:30,Premier League,Matchweek 37,Sat,Home,L,0,2,Burnley,...,Match Report,NaN,6.0,2.0,13.4,0.0,0,0,2019,Norwich City


In [ ]:
#Checking the number of matches for each team
df["Team"].value_counts()

Team
Manchester City             190
Manchester United           190
Everton                     190
Arsenal                     190
Brighton and Hove Albion    190
Crystal Palace              190
West Ham United             190
Wolverhampton Wanderers     190
Newcastle United            190
Tottenham Hotspur           190
Aston Villa                 190
Liverpool                   190
Chelsea                     190
Burnley                     152
Leicester City              152
Southampton                 152
Bournemouth                 114
Fulham                      114
Brentford                   114
Sheffield United            114
Leeds United                114
Norwich City                 76
Nottingham Forest            76
Watford                      76
Luton Town                   38
West Bromwich Albion         38
Name: count, dtype: int64

In [ ]:
#Converting the column names to lowercase
df.columns = [c.lower() for c in df.columns]

In [ ]:
#Displaying the first 5 lines of the dataframe
df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,2023,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,2023,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,2023,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,2023,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,29.0,13.0,16.4,1.0,0,0,2023,Manchester City


In [ ]:
#Saving the dataframe as a csv file
df.to_csv("matches_data.csv")